<a href="https://colab.research.google.com/github/advait1987/tf_gendervoiceclassify/blob/master/Gender_Voice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 79.9MB 371kB/s 
     |████████████████████████████████| 61kB 19.8MB/s 
     |████████████████████████████████| 419kB 44.2MB/s 
     |████████████████████████████████| 3.0MB 32.4MB/s 


In [3]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../tmp"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [0]:
url = 'https://github.com/advait1987/tf_gendervoiceclassify/raw/master/voice.csv'
dataframe = pd.read_csv(url)

## **Explore**

In [5]:
dataframe.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.0,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male


In [6]:
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
dataframe['label'] = gender_encoder.fit_transform(dataframe['label'])
dataframe.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.0,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1


In [7]:
print("Total number of samples: {}".format(dataframe.shape[0]))
print("Number of male: {}".format(dataframe[dataframe.label == 1].shape[0]))
print("Number of female: {}".format(dataframe[dataframe.label == 0].shape[0]))

Total number of samples: 3168
Number of male: 1584
Number of female: 1584


Split Data

In [8]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')
print (train.shape)
train.head()

2027 train examples
507 validation examples
634 test examples
(2027, 21)


,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
347,0.158272,0.060818,0.147618,0.107845,0.212523,0.104678,3.027477,13.167034,0.904559,0.482941,0.107251,0.158272,0.112817,0.039062,0.277778,1.016323,0.112305,3.999023,3.886719,0.352793,1
560,0.143504,0.073334,0.150426,0.086383,0.197713,0.111330,3.586009,28.346615,0.961517,0.734525,0.059947,0.143504,0.090472,0.017448,0.235294,0.462015,0.007812,2.335938,2.328125,0.203020,1
3088,0.162336,0.055594,0.168105,0.159551,0.187307,0.027756,7.151586,68.542788,0.875032,0.450097,0.165661,0.162336,0.149047,0.018622,0.208333,0.177590,0.073242,0.346680,0.273438,0.101648,0
2728,0.202353,0.038893,0.196491,0.182246,0.226947,0.044702,2.323108,9.261747,0.872001,0.249095,0.184702,0.202353,0.158921,0.048830,0.279070,1.812500,0.023438,10.898438,10.875000,0.166083,0
125,0.167705,0.070774,0.179442,0.097629,0.231771,0.134142,17.948488,416.183638,0.921257,0.519320,0.059944,0.167705,0.081219,0.015889,0.271186,0.073828,0.070312,0.210938,0.140625,0.051282,1


**Create an input pipeline using tf.data**
This will enable us to use feature columns as a bridge to map from the columns in the Pandas dataframe to features 
used to train the model. If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.

In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [10]:
# Test input pipeline
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['median'])
  print('A batch of targets:', label_batch )

Every feature: ['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt', 'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx']
A batch of ages: tf.Tensor([0.19550505 0.23341021 0.17976383 0.17980392 0.20691299], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([0 0 1 0 0], shape=(5,), dtype=int32)


**Choose which columns to use**

In [0]:
feature_columns = []

# numeric cols
for header in list(feature_batch.keys()):
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
#age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
#feature_columns.append(age_buckets)

# indicator cols
#thal = feature_column.categorical_column_with_vocabulary_list(
#      'thal', ['fixed', 'normal', 'reversible'])
#thal_one_hot = feature_column.indicator_column(thal)
#feature_columns.append(thal_one_hot)

# embedding cols
#thal_embedding = feature_column.embedding_column(thal, dimension=8)
#feature_columns.append(thal_embedding)

# crossed cols
#crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
#crossed_feature = feature_column.indicator_column(crossed_feature)
#feature_columns.append(crossed_feature)

**Create feature layer**

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
#Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

**Create, compile, and train the model**

In [26]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.95):
      print (logs.get('acc'))
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()   

model = tf.keras.Sequential([
  feature_layer,
  #tf.keras.layers.Conv1D(5,input_shape=(20,1), kernel_size=12, padding='same', activation='relu'),
  #tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=50,
          verbose = 1,
          callbacks=[callbacks])

Epoch 1/50
64/64 [==============================] - 2s 28ms/step - loss: 0.7711 - acc: 0.5065 - val_loss: 0.9157 - val_acc: 0.5582
Epoch 2/50
64/64 [==============================] - 2s 26ms/step - loss: 0.7986 - acc: 0.6094 - val_loss: 0.7125 - val_acc: 0.6923
Epoch 3/50
64/64 [==============================] - 2s 24ms/step - loss: 0.7459 - acc: 0.6465 - val_loss: 0.7498 - val_acc: 0.6963
Epoch 4/50
64/64 [==============================] - 2s 25ms/step - loss: 0.6867 - acc: 0.6801 - val_loss: 0.9071 - val_acc: 0.6923
Epoch 5/50
64/64 [==============================] - 2s 24ms/step - loss: 0.8644 - acc: 0.6966 - val_loss: 0.7875 - val_acc: 0.7278
Epoch 6/50
64/64 [==============================] - 2s 24ms/step - loss: 0.6227 - acc: 0.7186 - val_loss: 0.6676 - val_acc: 0.7850
Epoch 7/50
64/64 [==============================] - 2s 24ms/step - loss: 0.5966 - acc: 0.7503 - val_loss: 0.7494 - val_acc: 0.7613
Epoch 8/50
64/64 [==============================] - 2s 24ms/step - loss: 0.6388 - a

**Accuracy**

In [27]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

20/20 [==============================] - 0s 14ms/step - loss: 0.2167 - acc: 0.9448
Accuracy 0.94479495
